In [1]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
import keras

categories = np.load("category.npy").tolist()

# 전이학습 베이스 모델 
base_model = keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False

inputs = keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(128, activation='relu')(x)
outputs = keras.layers.Dense(len(categories), activation='softmax')(x)

model = keras.Model(inputs, outputs)

In [3]:
adam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint("best-model.keras", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
for i in range(49):
    data_path = f"data/data{i}.npy"
    target_path = f"data/target{i}.npy"
    train_data = np.load(data_path)
    target_data = np.load(target_path)
    total = train_data.shape[0]
    items = np.zeros((total, 224, 224), dtype=np.uint8)

    for j in range(total):
        items[j] = np.array(Image.fromarray(train_data[j]).resize((224, 224)))

    items = np.expand_dims(items, axis=3)
    items = np.repeat(items, 3, axis=3)
    
    # 훈련세트, 테스트세트 분리 
    train_input, test_input, train_target, test_target = train_test_split(
        items, target_data, test_size=0.1
    )

    # 훈련세트, 검증세트 분리 
    train_input, val_input, train_target, val_target = train_test_split(
        train_input, train_target, test_size=0.1
    )

    # 훈련 시작 
    model.fit(train_input, train_target, validation_data=(val_input, val_target), callbacks=[checkpoint_cb, early_stopping_cb])

    print("테스트 세트 점수:", model.evaluate(test_input, test_target))